# Process Data from Original Source (Fulton Files)

In [1]:
import os
import csv
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:.2f}'.format

FULTON_DIR = './data/raw_fulton/'
fulton_files = os.listdir(FULTON_DIR)

### Helper Functions

In [2]:
def contains_keyword(filename: str, keywords: list[str]) -> str:
    return any(keyword in filename for keyword in keywords)

def clean_and_cast_column(column: pd.Series, var_map: dict) -> pd.Series:
    to_dtype = var_map[column.name] 
    fill_val = None
    
    if to_dtype == "int" or to_dtype == "float":
        fill_val = 0
    elif to_dtype == "string":
        fill_val = ""
    else:
        raise ValueError(f"{to_dtype} is not a valid data type!")
    
    if ((to_dtype == "int" or to_dtype == "float")
        and (column.dtype == "string" or column.dtype == "object")):
        # Remove commas from number strings before converting
        column = column.astype("str").str.replace(",", "").astype('float')
    
    # Record number of filled nulls
    print(f"Number of nulls in column {column.name}: {column.isna().sum()}")
    
    column = column.fillna(fill_val)
    return column.astype(to_dtype)

## Create a single file with all LUC == 101 Fulton County Digest (Parcel) Data for all Years

In [3]:
keywords = ["DIGEST", "NF", "SF"]
digest_cols = {
    "Taxyr": "int",
    "Parid": "string",
    "Situs Adrno": "int",
    "Situs Adrdir": "string",
    "Situs Adrstr": "string",
    "Situs Adrsuf": "string",
    "Cityname": "string",
    "Luc": "string",
    "Calcacres": "float",
    "Own1": "string",
    "Own2": "string",
    "Owner Adrno": "int",
    "Owner Adradd": "string",
    "Owner Adrdir": "string",
    "Owner Adrstr": "string",
    "Owner Adrsuf": "string",
    "Cityname.1": "string",
    "Statecode": "string",
    "Zip1": "string",
    "Aprtot": "float",
    "D Yrblt": "int",
    "D Effyr": "int",
    "D Yrremod": "int",
    "Sfla": "float",
    "Rmbed": "int",
    "Fixbath": "int",
    "Calcacres": "float",
    "Heat": "int",
    "D Effyr": "int",
    "Extwall": "int",
    "Style": "string",
    "Rmtot": "int",
    "D Grade": "string",
    "Bsmt": "int"
}

desired_files = filter(lambda file: contains_keyword(file, keywords), fulton_files)

# Read desired files and only parse desired cols
# Need to ensure LUC is read in as a str so we can filter appropriately
desired_files_dfs = [
    pd.read_excel(
        FULTON_DIR + file,
        usecols=digest_cols,
        dtype={"Luc": "str"}
    ) for file in desired_files
]

In [4]:
# Concat selected digest files
# Select for LUC = 101 (SFH)
# Drop complete duplicates
digest_full = pd.concat(desired_files_dfs)
digest_full['Luc'] = digest_full['Luc'].astype('str')
digest_full = digest_full[digest_full['Luc'] == '101']

In [5]:
# After filtering for LUC, we can continue to cast other columns
rename = {
    "Taxyr": "TAXYR",
    "Parid": "PARID",
    "Cityname.1": "own_cityname",
    "Zip1": "own_zip",
    "Sfla": "sqft_living",
    "D Yrblt": "yr_built",
    "Rmbed": "beds",
    "Fixbath": "baths",
    "Calcacres": "acres",
    "Heat": "heat",
}
init_len = len(digest_full)

digest_full = digest_full.drop_duplicates()
print(f"Init len: {init_len}")
print(f"Number of dropped complete duplicates: {init_len - len(digest_full)}")
print(f"Final len: {len(digest_full)}")

# Drop nulls in important columns
drop_nulls_cols = [
    "Taxyr", "Parid", "Situs Adrno", "Situs Adrstr", "Luc", "D Yrblt",
    "Sfla", "Rmbed", "Fixbath", "Calcacres", "Heat", "Extwall",
    "Style", "Rmtot", "D Grade", "Bsmt", "Owner Adrstr"
]

init_len = len(digest_full)
digest_full = digest_full.dropna(subset=drop_nulls_cols)
print(f"Number of dropped nulls from important columns: {init_len - len(digest_full)}")

# Records nulls and set datatypes
for column in digest_full.columns:
    digest_full[column] = clean_and_cast_column(digest_full[column], digest_cols)
    
digest_full = digest_full.rename(columns=rename)

Init len: 3681749
Number of dropped complete duplicates: 896087
Final len: 2785662
Number of dropped nulls from important columns: 19690
Number of nulls in column Taxyr: 0
Number of nulls in column Parid: 0
Number of nulls in column Situs Adrno: 0
Number of nulls in column Situs Adrdir: 2763914
Number of nulls in column Situs Adrstr: 0
Number of nulls in column Situs Adrsuf: 188543
Number of nulls in column Cityname: 4745
Number of nulls in column Luc: 0
Number of nulls in column Calcacres: 0
Number of nulls in column Own1: 0
Number of nulls in column Own2: 2315696
Number of nulls in column Owner Adrno: 63862
Number of nulls in column Owner Adradd: 2761024
Number of nulls in column Owner Adrdir: 2704437
Number of nulls in column Owner Adrstr: 0
Number of nulls in column Owner Adrsuf: 209443
Number of nulls in column Cityname.1: 12
Number of nulls in column Statecode: 256
Number of nulls in column Zip1: 841
Number of nulls in column Aprtot: 0
Number of nulls in column Extwall: 0
Number 

In [6]:
# Quickly validate no data quality issues
# by looking at number of parcels per year
digest_full.groupby("TAXYR")['PARID'].count()

TAXYR
2010    207544
2011    207578
2012    207757
2013    208987
2014    209878
2015    211153
2016    201392
2017    214320
2018    215890
2019    217756
2020    219842
2021    221480
2022    222395
Name: PARID, dtype: int64

Note: It looks like a few parcels are lost between 2015 and 2016 (approx 10K) - this is potentially a flaw in Fulton data, not our own processing

## Create a File with all Fulton County Sales for all Years

In [7]:
keywords = ["STANDARDS SALES"]
sale_cols = {
    "Taxyr": "int",
    "Parid": "string",
    "Saledt": "string",
    "Luc": "string",
    "SALES PRICE": "float",
    "FAIR MARKET VALUE": "float",
    "DEED TYPE": "string",
    "Saleval": "string",
    "Costval": "string",
    "GRANTOR": "string",
    "GRANTEE": "string",
}

desired_files = filter(lambda file: contains_keyword(file, keywords), fulton_files)

desired_files_dfs = [
    pd.read_csv(
        FULTON_DIR + file,
        sep='\t',
        encoding='latin-1',
        usecols=sale_cols,
        quoting=csv.QUOTE_NONE,
        skipfooter=1,
        on_bad_lines="warn"
    ) for file in desired_files
]

C:\Users\Nick\AppData\Local\Temp\ipykernel_964\1001776804.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_964\1001776804.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_964\1001776804.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_964\1001776804.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipy

In [8]:
# Select for LUC = 101 (SFH)
sales_full = pd.concat(desired_files_dfs)
sales_full['Luc'] = sales_full['Luc'].astype('str')
sales_full = sales_full[sales_full['Luc'] == '101']

In [9]:
# Concat selected sales files
# Drop complete duplicates
# Record total number of sales
rename = {
    "Taxyr": "TAXYR",
    "Parid": "PARID",
}

init_len = len(sales_full)
sales_full = sales_full.drop_duplicates()
print(f"Init len: {init_len}")
print(f"Number of dropped complete duplicates: {init_len - len(sales_full)}")
print(f"Final len: {len(sales_full)}")

# Records nulls and set datatypes
for column in sales_full.columns:
    sales_full[column] = clean_and_cast_column(sales_full[column], sale_cols)
    
sales_full = sales_full.rename(columns=rename)

Init len: 275814
Number of dropped complete duplicates: 461
Final len: 275353
Number of nulls in column Taxyr: 0
Number of nulls in column Parid: 0
Number of nulls in column Luc: 0
Number of nulls in column Saledt: 0
Number of nulls in column SALES PRICE: 19
Number of nulls in column FAIR MARKET VALUE: 0
Number of nulls in column DEED TYPE: 2
Number of nulls in column Costval: 0
Number of nulls in column Saleval: 118
Number of nulls in column GRANTOR: 18
Number of nulls in column GRANTEE: 8


In [10]:
# Check distribution of data for quick validation
sales_full.groupby("TAXYR")['PARID'].count()

TAXYR
2011    26777
2012    22684
2013    26074
2014    11711
2015    11256
2016    14978
2017    13210
2018    28281
2019    29134
2020    29570
2021    28645
2022    33033
Name: PARID, dtype: int64

In [11]:
# Another data validation check
sales_full.groupby("TAXYR").describe()

SALES PRICE                                                          \
            count      mean        std  min       25%       50%       75%   
TAXYR                                                                       
2011     26777.00 162950.94  630129.89 0.00      0.00  56050.00 189900.00   
2012     22684.00 164550.67  424239.44 0.00      0.00  49500.00 200515.00   
2013     26074.00 166081.78  440832.36 0.00      0.00  47000.00 212000.00   
2014     11711.00 331640.54  351925.01 0.00  79000.00 255000.00 455000.00   
2015     11256.00 357182.81  392959.23 0.00 101425.00 275000.00 475000.00   
2016     14978.00 348266.13  401923.49 0.00  85000.00 258325.00 474500.00   
2017     13210.00 386344.64  386724.40 0.00 139000.00 294000.00 517500.00   
2018     28281.00 268587.37  931256.97 0.00      1.00 108000.00 360000.00   
2019     29134.00 522038.36 2227323.74 0.00      1.00 137000.00 379900.00   
2020     29570.00 303400.40 1085088.76 0.00      1.00 155000.00 379000.00   
2021     28645.00 363893.16 2274967.99 0.00      1.00 193000.00 435000.00   
2022     33033.00 530580.09 1934406.18 0.00     10.00 250000.00 537500.00   

                   FAIR MARKET VALUE                                        \
               max             count      mean       std     min       25%   
TAXYR                                                                        
2011   51718108.00          26777.00 210525.70 292278.99    0.00  42100.00   
2012   20500000.00          22684.00 241854.35 325947.29 1500.00  56600.00   
2013   17038094.00          26074.00 238072.86 323868.49    0.00  39500.00   
2014    6350000.00          11711.00 294653.60 320690.06  100.00  63900.00   
2015   13914719.00          11256.00 314046.58 324030.38 1390.00  74887.50   
2016   17455046.00          14978.00 346104.72 368631.02    0.00  91500.00   
2017    7200000.00          13210.00 306637.00 320307.43    0.00  75725.00   
2018  135635876.00          28281.00 307217.92 359431.09    0.00  75800.00   
2019   40120000.00          29134.00 328591.65 367677.75  100.00 107200.00   
2020   58800000.00          29570.00 364300.06 393939.77    0.00 133500.00   
2021  126940000.00          28645.00 422773.73 432387.60    0.00 173100.00   
2022  231058000.00          33033.00 478192.98 476555.17  800.00 205000.00   

                                       
            50%       75%         max  
TAXYR                                  
2011  115300.00 276200.00  8750000.00  
2012  142800.00 314800.00 13698600.00  
2013  142225.00 325900.00  6322800.00  
2014  226960.00 402150.00  6189700.00  
2015  244970.00 428770.00  6151600.00  
2016  257500.00 470000.00  6000000.00  
2017  225900.00 425000.00  6000000.00  
2018  198500.00 418000.00  7150000.00  
2019  207000.00 427975.00  7861300.00  
2020  243900.00 467500.00 11350000.00  
2021  303700.00 529900.00 15000000.00  
2022  342600.00 587500.00  9966300.00

## Geocode Digest and Sales Data

In [12]:
# Read in parcel boundary data from Fulton County 2022 record
geo_parcels = gpd.read_file("./data/fulton_parcels.geojson")
geo_parcels = geo_parcels[["ParcelID", "OBJECTID", "geometry"]]
geo_parcels = geo_parcels.rename(columns={"ParcelID": "PARID"})
# Ensure no duplicates for merging
print(len(geo_parcels[geo_parcels.duplicated(subset=["PARID"])]))

# Read in neighborhood boundaries and stats from Neighborhood Nexus
geo_atl_nsa = gpd.read_file("./data/atl_nsa.geojson")
geo_atl_nsa = geo_atl_nsa.rename(columns={"NEIGHBORHO": "neighborhood", "geometry": "nsa_boundary"})
geo_atl_nsa = geo_atl_nsa[["neighborhood", "nsa_boundary"]]
# Ensure no duplicates for merging
print(len(geo_atl_nsa[geo_atl_nsa.duplicated(subset=["neighborhood"])]))

# Validate identical CRS for spatial join
geo_parcels = geo_parcels.set_geometry("geometry")
geo_atl_nsa = geo_atl_nsa.set_geometry("nsa_boundary")
print(f"Validate CRS: {geo_parcels.crs == geo_atl_nsa.crs}")

0
0
Validate CRS: True


In [13]:
# Get city name for each parcel with a spatial join
geo_parcels = gpd.sjoin(geo_parcels, geo_atl_nsa, how="left", op="within")

c:\Users\Nick\Documents\code\equity-extraction\equity-extraction\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [14]:
# Merge parcel boundaries and cities with digest and salses data
# Just do this for record keeping purposes (e.g. how many can't be matched)
# Actual merging will be done in analysis file due to data format issues
digest_geo = digest_full.merge(geo_parcels, on="PARID", how="inner")
sales_geo = sales_full.merge(geo_parcels, on="PARID", how="inner")

print(f"Num of records not geo matched in digest: {len(digest_full) - len(digest_geo)}")
print(f"Num of records not geo matched in sales: {len(sales_full) - len(sales_geo)}")

Num of records not geo matched in digest: 6556
Num of records not geo matched in sales: 852


In [15]:
# Parcels and sales in ATL
digest_atl = digest_geo[digest_geo['neighborhood'].notna()]
sales_atl = sales_geo[sales_geo['neighborhood'].notna()]
print(f"Number of parcels in ATL: {len(digest_atl)}")
print(f"Number of sales in ATL: {len(sales_atl)}")

Number of parcels in ATL: 1002694
Number of sales in ATL: 110896


## Save Data

In [16]:
# Save parcel geometry data
OUTPUT_PATH = 'output/parcels_geo'
geo_parcels.to_csv(OUTPUT_PATH + '.csv', index=False)

# Save parcel data without geometry data so Parquet format works
OUTPUT_PATH = 'output/'
digest_full.to_parquet(OUTPUT_PATH + 'digest_full.parquet', index=False)
sales_full.to_parquet(OUTPUT_PATH + 'sales_full.parquet', index=False)